<a href="https://colab.research.google.com/github/ricglz/CE888_activities/blob/main/assignment/Data_augmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! [ ! -z "$COLAB_GPU" ] && pip install torch torchvision kaggleDownloader

# Preparations

----

In [2]:
from os import path, remove
from glob import glob
from tqdm import tqdm
import numpy as np
from sklearn.model_selection import train_test_split

from torchvision.datasets import ImageFolder
import torchvision.utils as t_utils
import torchvision.transforms as T

try:
    from kaggleDownloader import get_dataset
    from google.colab import drive
    in_colab = True
except ImportError:
    in_colab = False
import numpy as np

In [3]:
def get_training_path():
    if in_colab:
        drive_path = '/content/gdrive'
        drive.mount(drive_path, force_remount=False)
        return path.join(drive_path, 'MyDrive/Essex/Datasets/Flame/Training')
    return './Flame/Training'

In [4]:
classes = ['Fire', 'No_Fire']
training_path = get_training_path()
resize = T.Resize((224, 224))

In [5]:
def save_image(img, label, index, prefix):
    klass = classes[label]
    img_path = path.join(training_path, f'{klass}/{prefix}_{index}.png')
    t_utils.save_image(img, img_path)

# Resize images to the desired size

---

In [11]:
train_ds = ImageFolder(training_path, T.Compose([resize, T.ToTensor()]))

for index, (img_path, _) in enumerate(tqdm(train_ds.imgs)):
    img, _ = train_ds[index]
    t_utils.save_image(img, img_path)

100%|██████████| 76706/76706 [34:13<00:00, 37.35it/s]  


# Add new dataset to the existent one

----

One of the principal problems of the current dataset is that the images are basically a burst of shots of the same environment, this leads to the problem that the model learns to recognize the environment instead of recognizing fire

To avoid this we will use another [dataset that can be found in kaggle](https://www.kaggle.com/phylake1337/fire-dataset). Which may help the model to actually recognize the fire

In [ ]:
def add_kaggle_dataset():
    get_dataset('kaggle datasets download -d phylake1337/fire-dataset')
    transforms = T.Compose([resize, T.ToTensor()])
    extra_data = ImageFolder('/content/fire_dataset', transforms)
    for index, (img, label) in enumerate(tqdm(extra_data)):
        save_image(img, label, index, 'extra_dataset')

In [ ]:
add_kaggle_dataset()

# Balance datasets

----

In [ ]:
def clear_balanced():
    balance_imgs = glob(
        f'{training_path}/**/balance*.png', recursive=True)
    for balance_img in tqdm(balance_imgs):
        remove(balance_img)

In [ ]:
clear_balanced()

In [ ]:
def get_minor_klass(train_ds):
    targets = np.array(train_ds.targets)
    fire_data_count = np.count_nonzero(targets == 0)
    non_fire_data_count = np.count_nonzero(targets == 1)
    klass_counts = [fire_data_count, non_fire_data_count]
    minor_klass = np.argmin(klass_counts)
    minor_count, max_count = min(klass_counts), max(klass_counts)
    images_to_save = min(max_count - minor_count, minor_count)
    return minor_klass, images_to_save

In [ ]:
def balance_dataset():
    transforms = T.Compose([
      resize,
      T.ColorJitter(brightness=0.25, contrast=0.25),
      T.RandomRotation(degrees=5),
      T.RandomHorizontalFlip(),
      T.RandomVerticalFlip(),
      T.ToTensor(),
    ])
    train_ds = ImageFolder(training_path, transforms)
    minor_klass, images_to_save = get_minor_klass(train_ds)
    indexes_to_enhace = np.where(train_ds.targets == minor_klass)[0]
    assert train_ds.targets[indexes_to_enhace[0]] == minor_klass
    indexes_to_enhace = np.random.choice(indexes_to_enhace, images_to_save, replace=False)
    assert len(indexes_to_enhace) == images_to_save
    for save_img_index, index in enumerate(tqdm(indexes_to_enhace)):
        img, label = train_ds[index]
        save_image(img, label, save_img_index, 'balance')

In [ ]:
balance_dataset()

# Reduce the amount of data

In [6]:
def half_the_data():
    transforms = T.Compose([resize, T.ToTensor()])
    train_ds = ImageFolder(training_path, transforms)
    files = list(map(lambda a: a[0], train_ds.samples))
    _, erase_files = train_test_split(
        files, test_size=0.5, shuffle=True, stratify=train_ds.targets)
    for file_to_erase in tqdm(erase_files):
        remove(file_to_erase)

In [7]:
half_the_data()

100%|██████████| 38353/38353 [00:09<00:00, 4159.54it/s]


# Check final count

----

In [7]:
train_ds = ImageFolder(training_path)
targets = np.array(train_ds.targets)

In [8]:
fire_data_count = np.count_nonzero(targets == 0)
f'Fire data: {fire_data_count}'

'Fire data: 19176'

In [9]:
non_fire_data_count = np.count_nonzero(targets == 1)
f'Non-Fire data: {non_fire_data_count}'

'Non-Fire data: 19177'

In [10]:
f'Total: {fire_data_count + non_fire_data_count}'

'Total: 38353'